In [ ]:
%matplotlib inline

import matplotlib
import numpy as np
import matplotlib.pyplot as plt

# Introducing geopandas
In the next two weeks we will use the [geopandas](http://geopandas.org/) package to explore some key challenges of dealing with spatial data in the context of data science. That name probably requires at least a little bit of explanation. [Pandas](http://pandas.pydata.org/index.html) is an advanced package for manipulating tables of data.  The name is an approximate acronym for Python Data Analysis library.  Many of the capabilities pandas offers are available in the datascience package you have been using in foundation class, although pandas more advanced features mean that it is not an especially easy package to learn.

Anyway, geo-pandas combines pandas (data tables) with number of other packages, in particular, shapely (https://pypi.python.org/pypi/Shapely) which deals with geometry and fiona (https://pypi.python.org/pypi/Fiona) which handles spatial data formats.

So... let's import geopandas.

In [ ]:
import geopandas as gpd

## Open a data file
This is simple. The `read_file()` function will do the trick, and provided the file format makes sense, and is consistent with the file extension, you should have no issues.

In [ ]:
counties = gpd.read_file('ca-counties.shp')

Assign the file read result to a suitable variable name, and from there on you are dealing with a geopandas `GeoDataFrame` object, which allows us to examine the contents.  The `.head()` function will show us the first few rows of the data table.

In [ ]:
counties.head()

The important thing to notice here, is the **geometry** column in the table. It may not show up in the table view above. Let's take a look at it

In [ ]:
counties.geometry.head()

We can also use dictionary style notation to access the column.

In [ ]:
counties['geometry'].head()

What is a geometry? 

In [ ]:
counties.geometry[0]

Cool! You just made a map of Lassen County. Not a particularly useful map, but a map nevertheless.

The geometry column in a `GeoDataFrame` can store points, polygons, line segments or a variety of other `shapely` geometries, which can then be manipulated and managed in various ways according to our needs.

### Let's make some maps!

In [ ]:
counties.plot()

Kinda small. Try again.

In [ ]:
counties.plot(figsize=(6,9))

We can manipulate the look of this with a variety of other options in the `plot()` function. For example...

In [ ]:
counties.plot(figsize=(6,9), linewidth=0.25, edgecolor='w', facecolor='#999999')

You can find out more about the plotting options for basic maps in geopandas from [the documentation](http://geopandas.org/mapping.html) and also from the [matplotlib documentation](http://matplotlib.org/api/pyplot_api.html). 

For closer control over things, you need to a work at a lower level with the pyplot API. For example, to put county names on the map...

In [ ]:
fig = plt.figure()
ax = fig.add_subplot(111)
for polygon, name in zip(counties.geometry, counties.NAME):
    ax.annotate(xy=(polygon.centroid.x-50000, polygon.centroid.y), s=name)
counties.plot(ax=ax, linewidth=0.25, edgecolor='w', facecolor='#999999')

Hmmm... something not quite right there. The problem is that by working with the **matplotlib** low-level API, the plotting is no longer aware that the $(x, y)$ coordinates of the geometries are not just any old $(x, y)$ coordinates, but meaningful ones that need to be treated as such.  In particular, in most cases, they should be treated as scaled equally (in this case they are meters, from an arbitrary origin).  We enforce this by specifying that the plot should employ an equal aspect ratio.

In [ ]:
fig = plt.figure(figsize=(6,9))
ax = fig.add_subplot(111)
# Apply equal aspect ratio
ax.set_aspect('equal')
for polygon, name in zip(counties.geometry, counties.NAME):
    ax.annotate(xy=(polygon.centroid.x-50000, polygon.centroid.y), s=name)
counties.plot(ax=ax, linewidth=0.25, edgecolor='w', facecolor='#999999')

That's still not perfect, but it is a big improvement.

It is possible to make nice maps in geopandas, but to make finished publication quality maps, you will most likely want to export the map to a graphical file format for final tidy up in a package such as Inkscape or Illustrator. We aren't particularly concerned at the moment with publication quality maps so I'll just note the possibility and move on.

In [ ]:
## For example, you could do this, to make an SVG file
## fig.savefig('ca.svg')

## Mapping data
Our concern here is with *data* as much as with maps, so how can we map the data in that table we saw above?

We simply tell the `plot()` function what column in the data table to map.  Let's work initially with the `nPop` column which tells us the number of people recorded in the census in each county.

In [ ]:
counties.plot(column='nPop', linewidth=0.25, figsize=(6,9))

Not great. The problem here is the color scheme we are using (the default). Try this instead.

In [ ]:
counties.plot(column='nPop', cmap='Reds', linewidth=0.25, figsize=(6,9))

Hmmm. What's the problem now? It turns out counties are bad units for many kinds of analysis because they have radically different populations. We can see this by making a histogram of the county populations. 

In [ ]:
plt.hist(counties.nPop)

There are 53 counties in the state. 49 of them have populations under 1 million (well under that in fact), while Los Angeles county has over 9 million. That very skew distribution means we have to think carefully about how to map such data in a meaningful way. 

Take a look at the code below.

In [ ]:
fig = plt.figure(figsize=(15,8))
ax1 = plt.subplot(131)
ax1.set_aspect('equal')
counties.plot(ax=ax1, column='nPop', cmap='Blues', scheme='equal_interval', linewidth=0.25, legend=True)
ax2 = plt.subplot(132)
ax2.set_aspect('equal')
counties.plot(ax=ax2, column='nPop', cmap='Reds', scheme='quantiles', linewidth=0.25, legend=True)
ax3 = plt.subplot(133)
ax3.set_aspect('equal')
counties.plot(ax=ax3, column='nPop', cmap='Greens', scheme='fisher_jenks', linewidth=0.25, legend=True)

This makes three maps, each of them using a different *classification scheme* to determine how data values are mapped onto colors for display.  This is something you will look at in more detail later in the course, and it can have dramatic effects on the message conveyed by a map.

For now, in the cell below, experiment with making some maps of other columns in the data table.  Experiment with the `edgecolor`, `linewidth` and `facecolor` options.  Also change the color map `cmap` setting.  A list of the available color maps is [provided here](http://matplotlib.org/users/colormaps.html).

## Quicker plotting
You may have noticed that when we make three maps of California, GeoPandas `plot()` function gets quite slow.  There are technical reasons for this and a fix is in the pipeline for a future release of the package.  For now, I've made a hacky fix, that will allow you to plot things more quickly, but which might break (you have been warned).

It also can't produce legends or provide alternative classification schemes.

In [ ]:
import quickplot as qp

fig = plt.figure(figsize=(6,9))
qp.quickplot(counties, column='nPop', cmap='Greens', linewidth=0.25, edgecolor='k')

Note that key capabilities of geopandas `plot()` are not available in my hacky `quickplot()`, specifically `scheme` and `legend`.  That's not ideal, but at least when you are experimenting with code you can get maps more quickly with this approach.  This is important when we look at larger datasets like all the counties for the wholeof the US.

## Making a web map
Finally in this quick introduction, if you'd like to take a map from geopandas and make it into a web map, we can do that with a package called `mplleaflet`.

In [ ]:
import mplleaflet as mpl

qp.quickplot(counties.to_crs(epsg=4326), column='pWhite', cmap='Reds', edgecolor='k')
mpl.display()

There is a sneaky function call `to_crs()` in that code snippet, which relates to map projections. We'll look at that in more detail in the next session.